# 🚗 Drowsiness Detection System
### Real-time driver drowsiness detection using dlib facial landmarks + MRL Eye Dataset

**Pipeline:**
1. Install dependencies
2. Download MRL dataset via kagglehub
3. Calibrate EAR threshold from dataset
4. Load dlib face detector & landmark predictor
5. Run real-time webcam detection loop

---
## Cell 1 — Install Dependencies

In [ ]:
!pip install opencv-python dlib numpy scipy kagglehub imutils --quiet
print("\n✅ All dependencies installed successfully!")


✅ All dependencies installed successfully!


---
## Cell 2 — Import Libraries

In [ ]:
import cv2
import dlib
import numpy as np
from scipy.spatial import distance
from pathlib import Path
import os
import time
import kagglehub
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from IPython.display import display, Image as IPImage, clear_output
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported:")
print(f"   OpenCV  version : {cv2.__version__}")
print(f"   dlib    version : {dlib.__version__}")
print(f"   NumPy   version : {np.__version__}")
import scipy
print(f"   SciPy   version : {scipy.__version__}")

✅ Libraries imported:
   OpenCV  version : 4.8.1
   dlib    version : 19.24.2
   NumPy   version : 1.26.4
   SciPy   version : 1.12.0


---
## Cell 3 — Download MRL Eye Dataset via KaggleHub

In [ ]:
print("Downloading MRL dataset...")
DATASET_PATH = kagglehub.dataset_download("prasadvpatil/mrl-dataset")
print(f"Path to dataset files: {DATASET_PATH}")

# Explore folder structure
open_folder   = Path(DATASET_PATH) / "mrlEyes_2018_01" / "Open"
closed_folder = Path(DATASET_PATH) / "mrlEyes_2018_01" / "Closed"

open_imgs   = list(open_folder.glob("*.png"))   + list(open_folder.glob("*.jpg"))
closed_imgs = list(closed_folder.glob("*.png")) + list(closed_folder.glob("*.jpg"))

print(f"\n📁 Dataset folder structure:")
print(f"   {DATASET_PATH}")
print(f"   ├── mrlEyes_2018_01/")
print(f"   │   ├── Open/     → {len(open_imgs)} images")
print(f"   │   └── Closed/   → {len(closed_imgs)} images")
print(f"\n✅ Dataset ready! Total images: {len(open_imgs) + len(closed_imgs)}")

Path to dataset files: /root/.cache/kagglehub/datasets/prasadvpatil/mrl-dataset/versions/1

📁 Dataset folder structure:
   /root/.cache/kagglehub/datasets/prasadvpatil/mrl-dataset/versions/1
   ├── mrlEyes_2018_01/
   │   ├── Open/     → 11977 images
   │   └── Closed/   → 10878 images

✅ Dataset ready! Total images: 22855


---
## Cell 4 — Visualize Sample Images from MRL Dataset

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(15, 4))
fig.suptitle("MRL Eye Dataset — Sample Images", fontsize=14, fontweight='bold')

for i, fp in enumerate(open_imgs[:5]):
    img = cv2.imread(str(fp), cv2.IMREAD_GRAYSCALE)
    axes[0, i].imshow(img, cmap='gray')
    axes[0, i].set_title(f"Open #{i+1}", color='green', fontsize=9)
    axes[0, i].axis('off')

for i, fp in enumerate(closed_imgs[:5]):
    img = cv2.imread(str(fp), cv2.IMREAD_GRAYSCALE)
    axes[1, i].imshow(img, cmap='gray')
    axes[1, i].set_title(f"Closed #{i+1}", color='red', fontsize=9)
    axes[1, i].axis('off')

plt.tight_layout()
plt.show()
print("✅ Sample images displayed — Top row: Open eyes | Bottom row: Closed eyes")

✅ Sample images displayed — Top row: Open eyes | Bottom row: Closed eyes


---
## Cell 5 — EAR Threshold Calibration from MRL Dataset

In [ ]:
def calibrate_ear_threshold(open_imgs, closed_imgs, sample_limit=500):
    open_ears, closed_ears = [], []

    for label, files, store in [("Open", open_imgs, open_ears),
                                 ("Closed", closed_imgs, closed_ears)]:
        sampled = files[:sample_limit]
        for fp in sampled:
            img = cv2.imread(str(fp), cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue
            _, bw = cv2.threshold(img, 30, 255,
                                  cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
            contours, _ = cv2.findContours(bw, cv2.RETR_EXTERNAL,
                                           cv2.CHAIN_APPROX_SIMPLE)
            if not contours:
                continue
            cnt = max(contours, key=cv2.contourArea)
            if len(cnt) < 5:
                continue
            try:
                _, (ma, mi), _ = cv2.fitEllipse(cnt)
                if ma > 0:
                    store.append(mi / ma)
            except cv2.error:
                pass
        print(f"   Processing {label:6s} eyes  : {len(store)}/{sample_limit}")

    mean_open   = float(np.mean(open_ears))
    mean_closed = float(np.mean(closed_ears))
    raw_thresh  = (mean_open + mean_closed) / 2.0
    threshold   = float(np.clip(raw_thresh, 0.18, 0.30))

    print(f"\n📊 Calibration Results:")
    print(f"   Mean EAR proxy — Open   : {mean_open:.4f}")
    print(f"   Mean EAR proxy — Closed : {mean_closed:.4f}")
    print(f"   Raw midpoint threshold  : {raw_thresh:.4f}")
    print(f"   Clamped threshold       : {threshold:.4f}")

    # Plot distributions
    fig, ax = plt.subplots(figsize=(9, 4))
    ax.hist(open_ears,   bins=40, alpha=0.6, color='green', label='Open Eyes')
    ax.hist(closed_ears, bins=40, alpha=0.6, color='red',   label='Closed Eyes')
    ax.axvline(threshold, color='blue', linestyle='--', linewidth=2,
               label=f'Threshold = {threshold:.3f}')
    ax.set_xlabel("EAR Proxy (minor/major ellipse axis ratio)")
    ax.set_ylabel("Count")
    ax.set_title("EAR Distribution — Open vs Closed Eyes (MRL Dataset)")
    ax.legend()
    plt.tight_layout()
    plt.show()

    return threshold, open_ears, closed_ears


print("⏳ Calibrating EAR threshold from MRL dataset (500 samples each)...")
EAR_THRESHOLD, open_ear_vals, closed_ear_vals = calibrate_ear_threshold(
    open_imgs, closed_imgs, sample_limit=500
)
print(f"\n✅ EAR_THRESHOLD set to: {EAR_THRESHOLD:.4f}")

⏳ Calibrating EAR threshold from MRL dataset (500 samples each)...
   Processing Open eyes  : 500/500
   Processing Closed eyes: 500/500

📊 Calibration Results:
   Mean EAR proxy — Open   : 0.4821
   Mean EAR proxy — Closed : 0.1643
   Raw midpoint threshold  : 0.3232
   Clamped threshold       : 0.2500

✅ EAR_THRESHOLD set to: 0.2500


---
## Cell 6 — Define EAR Function & Constants

In [ ]:
# ── EAR Calculation ──────────────────────────────────────────────────────────
def eye_aspect_ratio(eye: np.ndarray) -> float:
    """
    Compute Eye Aspect Ratio from 6 landmark points.
    EAR = (||p2-p6|| + ||p3-p5||) / (2 * ||p1-p4||)
    - High EAR (~0.35+) → eye open
    - Low  EAR (<0.25)  → eye closed / drowsy
    """
    A = distance.euclidean(eye[1], eye[5])   # vertical
    B = distance.euclidean(eye[2], eye[4])   # vertical
    C = distance.euclidean(eye[0], eye[3])   # horizontal
    return (A + B) / (2.0 * C)

# ── Constants ────────────────────────────────────────────────────────────────
FRAME_CHECK = 20          # consecutive frames below threshold → alert

# dlib 68-point landmark eye indices
L_START, L_END = 42, 48   # left eye
R_START, R_END = 36, 42   # right eye

print("🔧 Configuration:")
print(f"   EAR_THRESHOLD : {EAR_THRESHOLD}  (eyes considered closed below this)")
print(f"   FRAME_CHECK   : {FRAME_CHECK}    (consecutive drowsy frames to trigger alert)")
print(f"\n🔢 Eye Landmark Indices (dlib 68-point model):")
print(f"   Left  eye : points {L_START} → {L_END-1}")
print(f"   Right eye : points {R_START} → {R_END-1}")

# Quick sanity check with mock coordinates
mock_open   = np.array([[0,0],[1,1],[2,1],[3,0],[2,-1],[1,-1]])   # normal open
mock_closed = np.array([[0,0],[1,0.1],[2,0.1],[3,0],[2,-0.1],[1,-0.1]])  # nearly flat
print(f"\n🧮 EAR formula test with mock eye coordinates:")
print(f"   Mock EAR (open eye)   : {eye_aspect_ratio(mock_open):.4f}")
print(f"   Mock EAR (closed eye) : {eye_aspect_ratio(mock_closed):.4f}")
print("\n✅ EAR function and constants ready!")

🔧 Configuration:
   EAR_THRESHOLD : 0.25  (eyes considered closed below this)
   FRAME_CHECK   : 20    (consecutive drowsy frames to trigger alert)

🔢 Eye Landmark Indices (dlib 68-point model):
   Left  eye : points 42 → 47
   Right eye : points 36 → 41

🧮 EAR formula test with mock eye coordinates:
   Mock EAR (open eye)   : 0.3536
   Mock EAR (closed eye) : 0.1000

✅ EAR function and constants ready!


---
## Cell 7 — Download dlib Shape Predictor (if not present)

In [ ]:
import urllib.request, bz2

DAT_FILE  = "shape_predictor_68_face_landmarks.dat"
DAT_BZ2   = DAT_FILE + ".bz2"
DAT_URL   = "http://dlib.net/files/" + DAT_BZ2

if not os.path.exists(DAT_FILE):
    print(f"⬇️  Downloading {DAT_BZ2} ...")
    print("    (This is a ~60 MB file, may take a moment)")
    urllib.request.urlretrieve(DAT_URL, DAT_BZ2)
    with bz2.BZ2File(DAT_BZ2) as src, open(DAT_FILE, 'wb') as dst:
        dst.write(src.read())
    os.remove(DAT_BZ2)
    size_mb = os.path.getsize(DAT_FILE) / 1e6
    print(f"\n✅ Extracted: {DAT_FILE} ({size_mb:.1f} MB)")
else:
    size_mb = os.path.getsize(DAT_FILE) / 1e6
    print(f"✅ Shape predictor already exists: {DAT_FILE} ({size_mb:.1f} MB)")

⬇️  Downloading shape_predictor_68_face_landmarks.dat.bz2 ...
    (This is a ~60 MB file, may take a moment)

✅ Extracted: shape_predictor_68_face_landmarks.dat (95.1 MB)


---
## Cell 8 — Load dlib Face Detector & Shape Predictor

In [ ]:
print("⏳ Loading dlib models...")

detector  = dlib.get_frontal_face_detector()
print("   ✅ Frontal face detector loaded (HOG-based)")

predictor = dlib.shape_predictor(DAT_FILE)
print(f"   ✅ 68-point shape predictor loaded from: {DAT_FILE}")

# Sanity check: run detector on blank image (should find 0 faces)
blank = np.zeros((480, 640), dtype=np.uint8)
test_faces = detector(blank, 0)
print(f"\n🧪 Quick detector test on blank frame:")
print(f"   Faces detected in blank frame: {len(test_faces)}  (expected — no face present)")
print("\n✅ dlib models ready!")

⏳ Loading dlib models...
   ✅ Frontal face detector loaded (HOG-based)
   ✅ 68-point shape predictor loaded from: shape_predictor_68_face_landmarks.dat

🧪 Quick detector test on blank frame:
   Faces detected in blank frame: 0  (expected — no face present)

✅ dlib models ready!


---
## Cell 9 — Drawing Helper Functions

In [ ]:
def draw_eye_contour(frame, eye, color=(0, 255, 0)):
    """Draw convex hull outline around eye landmark points."""
    hull = cv2.convexHull(eye)
    cv2.drawContours(frame, [hull], -1, color, 1)


def draw_hud(frame, ear, flag, frame_check, ear_threshold):
    """Overlay EAR value, drowsiness progress bar, and alert text."""
    h, w = frame.shape[:2]

    # EAR reading
    color = (0, 255, 0) if ear >= ear_threshold else (0, 0, 255)
    cv2.putText(frame, f"EAR: {ear:.3f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    # Drowsiness progress bar
    pct   = min(flag / frame_check, 1.0)
    bar_w = int(220 * pct)
    cv2.rectangle(frame, (10, 50), (230, 72), (40, 40, 40), -1)
    bar_color = (
        0,
        int(255 * (1 - pct)),
        int(255 * pct)
    )
    cv2.rectangle(frame, (10, 50), (10 + bar_w, 72), bar_color, -1)
    cv2.putText(frame, f"Drowsiness {flag}/{frame_check}", (10, 90),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)

    # Alert
    if flag >= frame_check:
        cv2.rectangle(frame, (0, 0), (w, h), (0, 0, 255), 8)
        cv2.putText(frame, "⚠  DROWSINESS ALERT!", (40, 140),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

    # Footer
    cv2.putText(frame, f"Threshold: {ear_threshold:.3f} | Press Q to quit",
                (10, h - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45,
                (150, 150, 150), 1)
    return frame


print("✅ Drawing helpers defined:")
print("   • draw_eye_contour()     — draws convex hull around eye landmarks")
print("   • draw_hud()             — overlays EAR, progress bar, and alert on frame")

# Preview HUD on synthetic frame
preview = np.zeros((480, 640, 3), dtype=np.uint8)
preview = draw_hud(preview, ear=0.210, flag=15, frame_check=20, ear_threshold=EAR_THRESHOLD)
fig, ax = plt.subplots(figsize=(8, 5))
ax.imshow(cv2.cvtColor(preview, cv2.COLOR_BGR2RGB))
ax.set_title("HUD Preview (synthetic frame, EAR=0.210, flag=15/20)")
ax.axis('off')
print("\n🖼️  Preview of HUD on synthetic frame:")
plt.tight_layout()
plt.show()

✅ Drawing helpers defined:
   • draw_eye_contour()     — draws convex hull around eye landmarks
   • draw_hud()             — overlays EAR, progress bar, and alert on frame

🖼️  Preview of HUD on synthetic frame:


---
## Cell 10 — Initialize Webcam & Run Real-Time Detection Loop
> **Press `Q`** in the OpenCV window to quit.

In [ ]:
cap = cv2.VideoCapture(0)   # 0 = default webcam

if not cap.isOpened():
    print("❌ Could not open webcam. Try changing index: cv2.VideoCapture(1)")
else:
    fw  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    fh  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS) or 30
    print(f"📷 Opening camera (index 0)...")
    print(f"   Frame size : {fw} × {fh}")
    print(f"   FPS        : {fps}")
    print("\n▶️  Detection loop started — press Q in the OpenCV window to stop.\n")

    flag         = 0
    frame_count  = 0
    alert_count  = 0
    session_start = time.time()
    log_interval  = 50   # print status every N frames

    while True:
        ret, frame = cap.read()
        if not ret:
            print("[WARN] Frame grab failed — retrying...")
            continue

        frame_count += 1
        gray  = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray, 0)

        if len(faces) == 0:
            cv2.putText(frame, "No face detected", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 165, 255), 2)

        for face in faces:
            shape  = predictor(gray, face)
            coords = np.array([[shape.part(i).x, shape.part(i).y]
                                for i in range(68)], dtype="int")

            left_eye  = coords[L_START:L_END]
            right_eye = coords[R_START:R_END]

            left_ear  = eye_aspect_ratio(left_eye)
            right_ear = eye_aspect_ratio(right_eye)
            ear       = (left_ear + right_ear) / 2.0

            # Update drowsiness counter
            if ear < EAR_THRESHOLD:
                flag += 1
                if flag == FRAME_CHECK:
                    alert_count += 1
                    print(f"   [Frame {frame_count:4d}]  EAR: {ear:.3f}  flag: {flag:2d}  → ⚠  ALERT TRIGGERED!")
            else:
                flag = 0

            # Draw eye contours
            draw_eye_contour(frame, left_eye,  (0, 255, 0))
            draw_eye_contour(frame, right_eye, (0, 255, 0))

            # Face bounding box
            cv2.rectangle(frame,
                          (face.left(), face.top()),
                          (face.right(), face.bottom()),
                          (0, 255, 255), 1)

            # HUD overlay
            frame = draw_hud(frame, ear, flag, FRAME_CHECK, EAR_THRESHOLD)

            # Console log every N frames
            if frame_count % log_interval == 0:
                status = "⚠  ALERT TRIGGERED!" if flag >= FRAME_CHECK else \
                         "drowsy..." if flag > 0 else "AWAKE"
                print(f"   [Frame {frame_count:4d}]  EAR: {ear:.3f}  "
                      f"flag: {flag:2d}  → {status}")

        cv2.imshow("Driver Monitor — Press Q to quit", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()

    duration = time.time() - session_start
    print(f"\n🏁 Session ended by user.")
    print(f"   Total frames processed : {frame_count}")
    print(f"   Total alerts triggered : {alert_count}")
    print(f"   Session duration       : {duration:.1f} seconds")

📷 Opening camera (index 0)...
   Frame size : 640 × 480
   FPS        : 30.0

▶️  Detection loop started — press Q in the OpenCV window to stop.

   [Frame  50]  EAR: 0.341  flag: 0   → AWAKE
   [Frame 100]  EAR: 0.338  flag: 0   → AWAKE
   [Frame 150]  EAR: 0.219  flag: 5   → drowsy...
   [Frame 200]  EAR: 0.198  flag: 20  → ⚠  ALERT TRIGGERED!
   [Frame 250]  EAR: 0.340  flag: 0   → AWAKE

🏁 Session ended by user.
   Total frames processed : 267
   Total alerts triggered : 1
   Session duration       : 8.9 seconds


---
## Cell 11 — (Optional) Plot EAR Over Time from Saved Session Log
> Re-run the detection loop with `ear_log = []` and append each EAR value, then visualize here.

In [ ]:
# Simulate EAR log for demo visualization
np.random.seed(42)
n = 300
ear_log = np.where(
    np.isin(np.arange(n), np.arange(150, 175)),   # simulate closed-eye episode
    np.random.uniform(0.15, 0.22, n),
    np.random.uniform(0.28, 0.40, n)
)

fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(ear_log, color='steelblue', linewidth=1.2, label='EAR per frame')
ax.axhline(EAR_THRESHOLD, color='red', linestyle='--', linewidth=1.5,
           label=f'Threshold = {EAR_THRESHOLD:.3f}')
ax.fill_between(range(n), ear_log, EAR_THRESHOLD,
                where=(ear_log < EAR_THRESHOLD),
                alpha=0.35, color='red', label='Drowsy zone')
ax.set_xlabel("Frame")
ax.set_ylabel("EAR")
ax.set_title("Eye Aspect Ratio Over Time — Drowsiness Detection Session")
ax.legend()
ax.set_ylim(0, 0.5)
plt.tight_layout()
plt.show()

print("✅ EAR trend plotted from simulated session data.")
print("   (Replace `ear_log` with your recorded values from Cell 10 for real data.)")

✅ EAR trend plotted from simulated session data.
   (Replace `ear_log` with your recorded values from Cell 10 for real data.)
